## Import library

In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime

import yfinance as yf
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

import sambo
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy

from backtesting.test import SMA, GOOG

# backtesting.set_bokeh_output(notebook=True)
            
import itertools 
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True

C:\Users\ASUS\AppData\Local\Temp\ipykernel_15928\1650128870.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
d:\Binance Tradebot\.venv\Lib\site-packages\backtesting\_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

## Import utils

In [2]:
from utils.loader import *
from utils.signals import *
from utils.trade import *
from utils.strategy import *

## Plotting sample

In [3]:
# now = datetime.today().strftime('%Y-%m-%d')

loader = DataLoader(ticker='AAPL', start='2022-06-01', end='2024-12-31', freq='1d', test_size=0.2)
loader.run()
# test     

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


## Strategy evaluation 

In [4]:
strategy = EmaCross
bt = BackTrader(data=loader.data, strategy=strategy, commission=0.002)

params = {
    'short_duration': 5,
    'long_duration': 10
}
bt.execute(data=bt.test_data, params=params, plot=True)
bt.trades.head()

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag,"Entry_EMA(C,5)","Exit_EMA(C,5)","Entry_EMA(C,10)","Exit_EMA(C,10)"
0,4,3,5,216.023956,220.344435,205.222758,220.344435,17.281917,0.020000,2024-07-01,2024-07-03,2 days,Long 0,213.241460,217.161538,212.883099,215.313088
1,4,33,36,220.784073,225.199754,209.744869,225.199754,17.662726,0.020000,2024-08-13,2024-08-16,3 days,Long 1,216.507692,221.938341,215.988450,219.591403
2,4,59,71,228.367386,223.807427,216.949017,232.934734,-18.239834,-0.019968,2024-09-19,2024-10-07,18 days,Long 2,222.232976,224.680519,221.678687,225.291204
3,4,73,77,229.035919,233.616638,217.584123,233.616638,18.322874,0.020000,2024-10-09,2024-10-15,6 days,Long 3,226.264248,229.968752,225.969532,228.413594
4,4,101,106,227.769577,232.324969,216.381098,232.324969,18.221566,0.020000,2024-11-18,2024-11-25,7 days,Long 4,226.223247,229.779267,226.032342,228.467808


## Cross-validation sample

In [8]:
grid_search = {
    'short_duration': range(2, 4),
    'long_duration': range(5, 11)
}

bt.cross_val(train_size=80, test_size=80, step_size=20, grid=grid_search)

{'short_duration': 3, 'long_duration': 9}
10033.598294947013
{'short_duration': 3, 'long_duration': 8}
9938.721666077634
{'short_duration': 3, 'long_duration': 10}
10057.012279318238
{'short_duration': 3, 'long_duration': 10}
10036.236591255523
{'short_duration': 3, 'long_duration': 9}
10016.985053404262
{'short_duration': 2, 'long_duration': 7}
10033.250587033213
{'short_duration': 3, 'long_duration': 8}
10007.143803366505
{'short_duration': 3, 'long_duration': 7}
10023.795251631538
{'short_duration': 2, 'long_duration': 7}
10036.228263345458
{'short_duration': 3, 'long_duration': 7}
10036.430423146867
{'short_duration': 2, 'long_duration': 5}
10006.877337632333
{'short_duration': 2, 'long_duration': 7}
10015.63303361134
{'short_duration': 3, 'long_duration': 7}
10001.592982451673
{'short_duration': 3, 'long_duration': 7}
10009.570650092403
{'short_duration': 3, 'long_duration': 8}
9994.317863189299
{'short_duration': 3, 'long_duration': 9}
9980.014939008754
{'short_duration': 3, 'lon

10012.437128606323